In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
# ! gdown 1IpRKfTLu9RILOlwtEsD-BbPhxaU6O8lS
! gdown 1uPMIFDrQ07rqyv6NOUwcSFVpBkZJ5FVG

Downloading...
From: https://drive.google.com/uc?id=1uPMIFDrQ07rqyv6NOUwcSFVpBkZJ5FVG
To: /content/Toxic_text_classifier_v2.keras
100% 156M/156M [00:06<00:00, 24.1MB/s]


In [ ]:
model = tf.keras.models.load_model('Toxic_text_classifier_v2.keras')
df = pd.read_csv('https://raw.githubusercontent.com/vzhou842/profanity-check/master/profanity_check/data/clean_data.csv')

In [ ]:
df = df[['text', 'is_offensive']]
df = df.rename(columns = {'is_offensive' : 'Is_Toxic'})
df = df.dropna()

In [ ]:
X = df['text']
y = df[df.columns[1:]].values

In [ ]:
from tensorflow.keras.layers import TextVectorization
vectorizer = TextVectorization(max_tokens = 100000,
                               output_mode='int')
vectorizer.adapt(X.values)

In [ ]:
input_text = vectorizer('in my opinion,you suck delete your instagram')
res = model.predict(np.array([input_text]))
res

1/1 [==============================] - 1s 834ms/step


array([[0.9993932]], dtype=float32)

In [ ]:
!pip install -q gradio

In [ ]:
import gradio as gr
def score_comment(comment):
    vectorized_comment = vectorizer([comment])
    results = model.predict(vectorized_comment)

    text = ''
    for idx, col in enumerate(df.columns[1:]):
        text += '{}: {}\n'.format(col, results[0][idx]>0.5)

    return text
interface = gr.Interface(fn=score_comment, title = "Toxic Text Classifier",
                         inputs=gr.components.Textbox(lines=2, placeholder='Comment to score'),
                        outputs='text')
interface.launch(share=True)